In [ ]:
%matplotlib inline


Learn algorithm and customize your input raster without writing it on disk
=============================================================================

This example shows how to customize your raster (ndvi, smooth signal...) in the 
learning process to avoi generate a new raster.




Import librairies
-------------------------------------------



In [ ]:
import numpy as np
from museotoolbox.learn_tools import learnAndPredict
from museotoolbox.cross_validation import RandomStratifiedKFold
from museotoolbox import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Load HistoricalMap dataset
-------------------------------------------



In [ ]:
raster,vector = datasets.historicalMap(low_res=True)
field = 'Class'

Create CV
-------------------------------------------



In [ ]:
SKF = RandomStratifiedKFold(n_splits=2,
                random_state=12,verbose=False)

Initialize Random-Forest and metrics
--------------------------------------



In [ ]:
classifier = RandomForestClassifier(random_state=12,n_jobs=1)

# 
kappa = metrics.make_scorer(metrics.cohen_kappa_score)
f1_mean = metrics.make_scorer(metrics.f1_score,average='micro')
scoring = dict(kappa=kappa,f1_mean=f1_mean,accuracy='accuracy')

Start learning
---------------------------
sklearn will compute different metrics, but will keep best results from kappa (refit='kappa')



In [ ]:
LAP = learnAndPredict(n_jobs=1,verbose=1)

Create or use custom function



In [ ]:
def bandRatio(X,bandToKeep=[0,2]):
    # this function get the first and the last band
    X=X[:,bandToKeep].reshape(-1,len(bandToKeep))
    return X

# add this function to learnAndPredict class
LAP.customizeX(bandRatio)

LAP.learnFromRaster(raster,vector,field,cv=SKF,
                    classifier=classifier,param_grid=dict(n_estimators=[10]),
                    scoring=scoring,refit='kappa')

Read the model
-------------------



In [ ]:
print(LAP.model)
print(LAP.model.cv_results_)
print(LAP.model.best_score_)

Get F1 for every class from best params
-----------------------------------------------



In [ ]:
for stats in LAP.getStatsFromCV(confusionMatrix=False,F1=True):
    print(stats['F1'])

Get each confusion matrix from folds
-----------------------------------------------



In [ ]:
for stats in LAP.getStatsFromCV(confusionMatrix=True):
    print(stats['confusionMatrix'])

Save each confusion matrix from folds
-----------------------------------------------



In [ ]:
LAP.saveCMFromCV('/tmp/testMTB/',prefix='RS50_')

Predict map
---------------------------



In [ ]:
LAP.predictRaster(raster,'/tmp/classification.tif',
                  confidence='/tmp/confidence.tif',
                  confidencePerClass='/tmp/confidencePerClass.tif')

Plot example



In [ ]:
from matplotlib import pyplot as plt
import gdal
src=gdal.Open('/tmp/classification.tif')
plt.imshow(src.GetRasterBand(1).ReadAsArray(),cmap=plt.get_cmap('tab20'))
plt.axis('off')
plt.show()